In [ ]:
# Setup

import os
import sys
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.advanced_activations import Quorum, ELU
from keras.datasets import mnist
from keras.utils import np_utils
from keras import backend as K
from keras.activations import relu, tanh
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD

af = "pap" #pap, relu, thresh, notrain
init_fn = "glorot_normal"

def pushln(line):
    sys.stdout.write(line)
    sys.stdout.flush()
    
def step(X):
    return K.switch(X <= 0, 0, 1)

def elu(X, alpha=1):
    pos = K.relu(X)
    neg = (X - abs(X)) * 0.5
    return pos + alpha * (K.exp(neg) - 1.)

def prepare_input_data(X_train, X_test):
    X_train = X_train.reshape(X_train.shape[0], 784)
    X_test = X_test.reshape(X_test.shape[0], 784)
    print X_train.shape
    X_train = X_train.astype('float32') / 255.0
    X_test = X_test.astype('float32') / 255.0
    return X_train, X_test

def prepare_output_data(y_train, y_test):
    y_train = np_utils.to_categorical(y_train)
    y_test = np_utils.to_categorical(y_test)
    return y_train, y_test

def get_af(name):
    af = name.lower()
    
    if af == "relu":
        return Activation('relu')
    elif af == "elu":
        return ELU()
    elif af == "pap":
        return Quorum([relu, step])
    elif af == "thresh":
        return Quorum([relu, step], threshold=0.5)
    elif af == "thresh2":
        return Quorum([elu, relu, tanh])
    elif af == "notrain":
        return Quorum([relu, step], trainable=False)
    else:
        raise RuntimeError("Unrecognized activation function: {}".format(name))

def add_dense_layer(model, num_nodes, input_shape=None):
    if input_shape:
        model.add(Dense(num_nodes, input_shape=input_shape, init=init_fn))
    else:
        model.add(Dense(num_nodes, init=init_fn))
    model.add(get_af(af))
        
# Script

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = prepare_input_data(X_train, X_test)
y_train, y_test = prepare_output_data(y_train, y_test)

model = Sequential()
add_dense_layer(model, 28, input_shape=(784,))
for x in range(10):
    add_dense_layer(model, 28)
model.add(Dense(10))
model.add(Activation('softmax'))

pushln("Compiling model...")
sgd = SGD(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=sgd)
pushln("finished.\n")
mc = ModelCheckpoint("mnist_weights_"+af, save_best_only=True, monitor='val_acc', verbose=1)
pushln("Beginning training...\n")
hist = model.fit(X_train, y_train, batch_size=128, verbose=1,
                        nb_epoch=500, show_accuracy=True,
                        validation_data=(X_test, y_test), callbacks=[mc])

(60000, 784)
Compiling model...Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
Not threshold!
finished.
Beginning training...
Train on 60000 samples, validate on 10000 samples
Epoch 1/500
60000/60000 [==============================] - 3s - loss: 1.0322 - acc: 0.7474 - val_loss: 0.4752 - val_acc: 0.8473
Epoch 00000: val_acc improved from -inf to 0.84730, saving model to mnist_weights_pap
Epoch 2/500
6